Exercises on pages 216-217.

Which Linear Regression training algorithm can be used on a training set that has millions of features?

Suppose the features in your traning set have very different scales.  Which algorithms might suffer from this, and how?  What can be done to mitigate this effect?

Can Gradient Descent get stuck in a local minimum when training a Logistic Regression model?

Do all Gradient Descent algorithms lead to the same model, provided that you run them long enough?

Suppose you use Batch Gradient Desecent and then plot the validation error at every epoch.  If you notice that the validation error consistently increases, what is the likely cause?  How can you fix this?

Is it a good idea to stop Mini-Batch Gradient Descent immediately upon the validation error increasing?

Which Gradient Descent algorithm (among those discussed in the chapter) will reach the vicinity of the optimal solution the fastest?  Which will actuallyconverge?  How can you make others converge as well?

Suppose you are using Polynomial Regression. You plot the learning curves and notice that there is a large gap between the training error and the validation error.  What is happening?  What are three ways to solve this problem?

Suppose you are using Ridge Regression and you notice that the training and validations errors are both similar and consistently high.  Is the model suffering from bias or variance?  Should you increase the regularization parameter $\alpha$, or reduce it?

Why would one use...

- ...Ridge Regression (i.e. as opposed to non-regularized Linear Regression)?
- ...LASSO instead of Ridge Regression?
- ...Elastic Net instead of LASSO?

Suppose you want to classify pictures as either outdoor or indoor, and either daytime or nighttime.  Should you implement two Logistic Regression classifiers or one Softmax Regression classifier?

Implement Batch Gradient Descent with early stopping for Softmax Regression, without resorting to `sklearn`.

Solutions on page 952.